 ## HUMBIO51 Assignment Week 5

<ol>
 <li> <a href=#wc>Use unix commands to count the number of lines in a file. </a></li>
 <li> <a href=#GORilla>Use GORilla to perform GO-term enrichment analysis. </a></li>
 <li> <a href=#GeneExpressionIntro>Visualize gene expression variation across cell types and individuals from the GTEx project</a></li>
 <li><a href=#RNASeqDataFormat>Use unix "head" and "cat" commands to merge and split files.</a></li>
 <li><a href=#RNASeqDataFormat>Construct contingency tables to compute GO-term enrichment likelihoods.</a></li>
 <li><a href=#RNASeqDataFormat>Write a Python expression for Fisher's exact test.</a></li>
 <li><a href=#RNASeqDataFormat>Apply Fisher's exact test to compute the probability of observing a particular contingency table.</a></li>


## Question 1
We would like to see how the expression of genes associated with GO term "[GO:0006915](http://amigo.geneontology.org/amigo/term/GO:0006915)" varies across different organs. We will use kmeans clustering to examine this. 

In [5]:
%%capture

%matplotlib inline
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/jovyan/humbio51/helpers')
from kmeans_helpers import * 

In [6]:
#Change your working directory to Week_5
import os
os.chdir('Week_5')

In [ ]:
!ls

### Question 1a
Load the RNAseq data expression values stored in file **/data/datasets/RNAseq/rnaseq_normalized.tsv** to the variable **rnaseq_data**.

In [ ]:
!ls /data

In [ ]:
## YOUR CODE HERE
import pandas as pd
rnaseq_data = pd.read_table(
      filepath_or_buffer='/data/datasets/RNAseq/rnaseq_normalized.tsv', 
      header=0,
      index_col=0)

### Question 1b

Recall that the rnaseq_data matrix has gene ids (rather than gene names) in the columns. We have converted the gene names in the file **GO_0006915.txt** to gene id's in the file **GO_0006915.ids.txt**. Extract the rows corresponding to the gene list in **GO_0006915.ids.txt** from **rnaseq_data** and store the subset of gene expression values in the variable **rnaseq_data_subset**. 

In [ ]:
## YOUR CODE HERE 
## HINT: read the gene ids in the file "GO_0006915.ids.txt" into a list. 
## This can be done using .read().strip().split('\n')
## HINT user the "iloc" command on the rnaseq_data object to extract the list of gene ids  

ids=open("GO_0006915.ids.txt",'r').read().strip().split('\n')
rnaseq_data_subset=rnaseq_data.loc[ids]

### Question 1c 
Perform k-means clustering on **rnaseq_data_subset** along the samples axis 

(hint: consider whether or not the matrix should be transposed to perform clustering along the samples)

Try values of k= 2,3,4. 
Which value of k is most effective in separating samples from clusters? 

In [ ]:
## YOUR CODE HERE 
for k in [3,4,5]:
    clusters=scikit_PCAandkmeans(rnaseq_data_subset.transpose(),
                       k,
                       xlabel='PC1',
                       ylabel='PC2',
                       plottitle='K-means clustering on the samples in the RNA-seq dataset (PCA-reduced data)'
                      )

### Question 1d 
Draw a heatmap of the rnaseq_data_subset. 

In [ ]:
import seaborn as sns 
sns.set(color_codes=False)

## YOUR CODE HERE ## (Hint: follow example from class 9)
plot_heatmap(rnaseq_data_subset)

Does the GO term split the samples into distinct clusters? 

ANSWER: 
    

## Question 2

We will take a close look in this question at how the p-values for GO-term enrichments are calculated by filling in a contingency table. Refer to the slides from class 10 for helpful tips on generating contingency tables and calculating p-values.


### QUESTION 2a 
We have downloaded the list of genes associated with GO term [GO:0008380](http://amigo.geneontology.org/amigo/term/GO:0008380). 
These are located in the Week_5 directory as GO_0008380.txt. 

These wil be the "Genes in GO term set" for the contingency table. 

Write out the number of genes that are in the G0_0008380 text file using unix commands.

In [ ]:
##YOUR CODE HERE
!wc -l GO_0008380.txt

### QUESTION 2b

To make a file for the "Genes in User set", we are going to add two hundred randomly selected genes (n = 200) to half of the GO term set.   

First use unix commands to print the first half of **GO_0008380.txt** (round down if there are an odd number of genes). Save the genes to a file called: 
**GO_0008380_half.txt** 

In [ ]:
##YOUR CODE HERE 
!wc -l GO_0008380.txt
427/2
!head -n 213 GO_0008380.txt > GO_0008380_half.txt

### QUESTION 2c

Now we will add 200 random genes to the file that was created in 2b. Below, we have defined the **add_random_genes** function: 

In [ ]:
def add_random_genes(target_gene_list,background_gene_list,n,outf): 
    """
    target_gene_list is a list containing the genes in your gene set of interest 
    background_gene_list is a list containing the genes in the background (i.e. hg19.names.txt)
    n is the number of genes from the background that should be randomly inserted into the target_gene_list 
    outf is the name of the output file for the gene set with the additional random genes 
    """
    #We import the numpy random  library to create a random number generator for selecting random genes from the background
    from numpy import random 
    
    #select n random genes from the background gene_list, without replacement 
    randomized=random.choice(background_gene_list,n,replace=False)

    #create a new target gene list by adding the random genes to the target gene list 
    import itertools 
    randomized_list=itertools.chain(target_gene_list,randomized )
    
    #open the output file
    out=open(outf,'w')
    
    #write the randomized target list to the output file 
    out.write('\n'.join(randomized_list))
    

Run the add_random_genes function on **GO_0008380_half.txt** with n = 200. For the background gene list you can use the hg19.names.txt file. 

Hint: To run the function, you will need to open the GO_0008380_half.txt and hg19.names.txt files. Instead of removing linebreaks like we have in the past, split the file at line breaks using .split('/n') eg:  

file=open(filename,'r').read().split('\n')

Save the resulting gene set in the file **randomized_genes.txt** 

In [ ]:
## YOUR CODE HERE 
target_gene_list=open("GO_0008380_half.txt").read().split('\n')
background_gene_list=open("hg19.names.txt").read().split('\n')
n=200 
outf="randomized_genes.txt"
add_random_genes(target_gene_list,background_gene_list,n,outf)

### Question 2d

For question 2e, you will fill in a contingency table using: 

* GO_0008380.txt as the "Genes in GO term set". 
* randomized_genes.txt  as the "Genes in User set" 
 
We have written the function below to help you calculate the intersection of "Genes not in User's Gene Set" and "Genes not in GO Term set". This function uses Python [sets](https://docs.python.org/2/library/sets.html), which are similar to lists, but differ in two ways: 
* the elements in a set are not ordered 
* all elements in a set are unique 

Use the "get_genes_not_in_user_set_or_go_term_set" function to calculate the number at the intersection of "Genes not in User's Gene Set" and "Genes not in GO Term set". Store this number in the variable "genes_not_in_either_set"

Remember that you will need to start off by opening the three files:
* GO_0008380.txt    
* randomized_genes.txt 
* hg19.names.txt 

It will be helpful to use the split command similarily to in question 2c above. 

In [ ]:
def get_genes_not_in_user_set_or_go_term_set(genes_in_go_term_set,genes_in_user_set,genes_in_background):
    """
    genes_in_go_term_set is a list containing names of genes associated with the GO term. 
    genes_in_user_set is a list containing the names of genes in the user's set. 
    genes_in_background is a list containing the names of genes in the background set. 
    """
    #convert all lists to sets to remove duplicated elements 
    genes_go=set(genes_in_go_term_set)
    genes_user=set(genes_in_user_set)
    genes_background=set(genes_in_background)
    
    #subtract the set of genes in the GO term set from the set of genes in the background 
    background_minus_go=genes_background - genes_go 
    
    #subtract the set of genes in the user set from "background_minus_go" 
    background_minus_go_minus_user=background_minus_go - genes_user 
    
    #return a list containing "background_minus_go_minus_user" 
    return list(background_minus_go_minus_user)



In [ ]:
#Use get_genes_not_in_user_set_or_go_term_set to calculate the intersection of 
#"Genes not in User's Gene Set" and "Genes not in GO Term set".

##YOUR CODE HERE ##
genes_in_go_term_set=open("GO_0008380.txt",'r').read().split('\n')
genes_in_user_set=open("randomized_genes.txt",'r').read().split('\n')
genes_in_background=open("hg19.names.txt",'r').read().split('\n')
##YOUR CODE HERE ##
genes_not_in_either_set=get_genes_not_in_user_set_or_go_term_set(genes_in_go_term_set,genes_in_user_set,genes_in_background)
print(len(genes_in_go_term_set))
print(len(genes_in_user_set))
print(len(genes_in_background))
print(len(genes_not_in_either_set))

### Question 2e
Now, use the value you obtained in the cell above to help fill in the contingency table: 

|    Contingency Table     | Genes in User's Gene Set | Genes not in User's Gene Set | Total       |
|--------------------------|--------------------------|------------------------------|-------------|
| Genes in GO Term set     | YOUR ANSWER              | YOUR ANSWER                  | YOUR ANSWER |
| Genes not in GO Term set | YOUR ANSWER              | YOUR ANSWER                  | YOUR ANSWER |
| Total                    | YOUR ANSWER              | YOUR ANSWER                  | YOUR ANSWER |

### Question 2f 

Recall from class 10 that Fischer's exact test can be used to calculate the probability of observing a given arrangement of numbers in a contingency table: 

![Fisher's exact test](../Images/HW_5_FishersTest.png)

Fill in the skeleton code below to derive a formula for Fisher's exact test. 
Hint: We provide a helper function **n_choose_k** that you should incorporate in your code. 

In [ ]:
import math
def n_choose_k(n,k): 
    """
    n = total number of possible outcomes 
    k = number of outcomes to select at random 
    """
    from decimal import Decimal
    #We use the Decimal class to help us compute with very large numbers. 
    #Factorials get very big quickly! 
    f=math.factorial 
    return Decimal(f(n))/Decimal(f(k))/Decimal(f(n-k))


#Complete this function with your code 
def fishers_exact_test(genes_in_users_gene_set,
                      genes_in_users_gene_set_and_go_term,
                      genes_not_in_users_gene_set,
                      genes_not_in_users_gene_set_and_in_go_term,
                      total,
                      genes_in_go_term): 
    p_value=None 
    genes_in_users_gene_set=1.0*genes_in_users_gene_set #we convert the integer to a double
    #FILL IN THE CODE HERE TO COMPUTE THE PVALUE
    p_value=(n_choose_k(genes_in_users_gene_set,genes_in_users_gene_set_and_go_term)*n_choose_k(genes_not_in_users_gene_set,genes_not_in_users_gene_set_and_in_go_term))/n_choose_k(total,genes_in_go_term)
    return p_value 

In [ ]:
## Execute the fishers_exact_test function to compute the probability of 
## observing the arrangement of values in your contingency table. 
## NOTE: The code is computing with very large numbers, so it may take up to 1 minute to run. 

## YOUR CODE HERE 

prob=fishers_exact_test(413,227,58168,200,58581,427)
print(prob)

### QUESTION 3

### QUESTION 3a

Use the [GORilla tool](http://cbl-gorilla.cs.technion.ac.il/) to identify Biological Process terms that are significantly enriched for the GO:0008380 GO-term gene list.
* The background gene set is the set of all human genes, stored in: hg19.names.txt in the Week_5 assignment folder*. 
* We consider a GO term to be significantly enriched if the FDR q-value is less than 0.05 

What description matches the GO term "GO:0008380"? 

ANSWER: 
RNA Splicing

What is the corrected p-value (FDR q-value) for this GO term? 

ANSWER:
0.00


### QUESTION 3b

Let's now find the enriched set of GO terms programmatically using a Python script. This approach should be less tedious than uploading multiple lists of genes to GORilla for analysis in the browser. 

To perform the analysis programatically, we will use the [goatools](https://github.com/tanghaibao/goatools) Python library. The goatools library provides a script called **find_enrichments.py** to find enriched GO terms in a list of genes. Let's examine the syntax of this script: 

In [7]:
from goatools import * 

In [2]:
!find_enrichment.py

fisher module not installed.  Falling back on scipy.stats.fisher_exact
usage: 
python find_enrichment.py study.file population.file gene-association.file

This program returns P-values for functional enrichment in a cluster of study
genes using Fisher's exact test, and corrected for multiple testing (including
Bonferroni, Holm, Sidak, and false discovery rate).

About significance cutoff:
--alpha: test-wise alpha; for each GO term, what significance level to apply
        (most often you don't need to change this other than 0.05 or 0.01)
--pval: experiment-wise alpha; for the entire experiment, what significance
        level to apply after Bonferroni correction

       [-h] [--annofmt {gene2go,gaf,gpad,id2gos}] [--taxid TAXID]
       [--alpha ALPHA] [--pval PVAL] [--pval_field PVAL_FIELD]
       [--outfile OUTFILE] [--ns NS] [--id2sym ID2SYM] [--sections SECTIONS]
       [--outfile_detail OUTFILE_DETAIL] [--compare] [--ratio RATIO]
       [--prt_study_gos_only] [--indent] [--obo OBO] 

There are 3 required arguments for the script: 
    
    1. The dataset (list of genes in Week_5/GO_0008380.txt) 
    2. The background (list of genes in Week_5/hg19.names.txt) 
    3. A file that associates gene names to GO terms (Week_5/hg19.assocs). We generated this file in advance -- It contains gene names in column 1 and GO terms associated with this gene in column 2. 

There are also 2 optional inputs to the script that we will find useful: 
    1. --outfile
    The output file to save enriched GO terms 
    2. --pval_field=fdr_bh 
    This argument indicates that the script should only return GO Terms enriched with false discovery rate < 0.05. 
    
Use this information to execute the script **find_enrichment.py** and store the output in a file **Week_5/GO_0008380.enrichments.tsv**

To execute the find_enrichment.py script you can use !find_enrichment.py followed by the arguments. 

In [8]:
#Execute the find_enrichment.py script for G0_0008380
! find_enrichment.py GO_0008380.txt hg19.names.txt hg19.assocs --outfile GO_0008380.enrichments.tsv --pval_field=fdr_bh

fisher module not installed.  Falling back on scipy.stats.fisher_exact
go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms
HMS:0:00:02.109899 315,594 annotations READ: hg19.assocs 
Study: 427 vs. Population 58581


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 29% 16,878 of 58,581 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 31% 17,901 of 58,581 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 29% 16,821 of 58,581 population items found in association

Run BP Gene Ontology Analysis: current study set of 427 IDs ...100%    413 of    413 study items found in association
 97%    413 of    427 study items found in population(58581)
Cal

Use the Unix head command to examine the format of the file "GO_0008380.enrichments.tsv"

In [9]:
## YOUR CODE HERE ## 
!head GO_0008380.enrichments.tsv

# GO	NS	enrichment	name	ratio_in_study	ratio_in_pop	p_uncorrected	depth	study_count	p_bonferroni	p_sidak	p_holm	p_fdr_bh	study_items
GO:0008380	BP	e	RNA splicing	352/413	352/58581	0.0	8	352	0.0	0.0	0.0	0.0	ACIN1, AFF2, AKAP17A, AKAP8L, ALYREF, AQR, ARL6IP4, BCAS2, BRDT, BUD13, BUD31, C1QBP, CACTIN, CASC3, CCAR1, CCAR2, CD2BP2, CDC40, CDC5L, CDK12, CDK13, CELF3, CELF4, CHERP, CIR1, CLASRP, CLP1, CPSF1, CPSF2, CPSF3, CPSF4, CPSF7, CRNKL1, CSTF1, CSTF2, CSTF2T, CSTF3, CTNNBL1, CWC15, CWC22, CWC25, CWC27, DBR1, DCPS, DDX1, DDX17, DDX23, DDX39A, DDX39B, DDX41, DDX42, DDX46, DDX47, DDX5, DHX15, DHX16, DHX35, DHX38, DHX40, DHX8, DHX9, DNAJC8, ECD, EFTUD2, EIF4A3, ELAVL1, ELAVL2, ERN1, ESRP1, ESRP2, FAM172A, FIP1L1, FMR1, FRG1, FUS, GCFC2, GEMIN2, GEMIN5, GEMIN6, GEMIN7, GPATCH1, GPKOW, GTF2F1, GTF2F2, HABP4, HNRNPA0, HNRNPA1, HNRNPA1L2, HNRNPA2B1, HNRNPA3, HNRNPC, HNRNPD, HNRNPF, HNRNPH1, HNRNPH2, HNRNPH3, HNRNPK, HNRNPL, HNRNPM, HNRNPR, HNRNPU, HNRNPUL1, HSPA8, HTATSF1, IK, ISY1, IVNS1ABP, I

Use the unix grep command to search for the GO term "GO:0008380" in the file GO_0008380.enrichments.tsv


In [ ]:
## YOUR CODE HERE ## 
! grep "GO:0008380" GO_0008380.enrichments.tsv 

In [14]:
! grep "GO:0008380" GO_0008380.enrichments.tsv > tmp.txt
! cut -f13 tmp.txt 

0.0


What is the FDR value of GO:0008380? (Hint: the FDR value is stored in the column "p_fdr_h", you should be able to see the order of the columns in the ouput from the head command)

ANSWER: 0.0

### QUESTION 3c 
What would happen to the p-values in the GO term enrichment analysis if only half of the genes in the gene set were available? Let's do this experiment and find out! 

Run the goatools analysis for GO_0008380_half.txt and store the output in a file **Week_5/GO_0008380_half.enrichments.tsv**

In [ ]:
## YOUR CODE HERE ## 
! find_enrichment.py GO_0008380_half.txt hg19.names.txt hg19.assocs --outfile GO_0008380_half.enrichments.tsv --pval_field=fdr_bh

Use the unix grep command to search for the GO term "GO:0008380" in the file GO_0008380_half.enrichments.tsv

In [ ]:
## YOUR CODE HERE ## 
! grep "GO:0008380" GO_0008380_half.enrichments.tsv

What is the FDR value of GO:0008380 in this new gene set?  (Hint: the FDR value is stored in the column "p_fdr_h", you should be able to see the order of the columns in the ouput from the head command)
    
ANSWER: 0.0
    

### QUESTION 3d

What will happen if we merge the two gene lists together? Will that change the p-values for the enriched GO terms? 

Use the unix **cat** command to create a file in called "merged_gene_lists.txt" that contains all the genes in "GO_0008380.txt" and "GO_0006915.txt". 



In [ ]:
## YOUR CODE HERE ## 
! cat GO_0006915.txt GO_0008380.txt > merged_gene_lists.txt

Re-run the goatools analysis for the merged file using the **find_enrichment.py** script. 

In [ ]:
## YOUR CODE HERE ## 
! find_enrichment.py merged_gene_lists.txt hg19.names.txt hg19.assocs --outfile GO_0008380_merged.enrichments.tsv --pval_field=fdr_bh

Run the unix grep command to search for GO terms "GO:0008380" in the output file from the find_enrichment.py script that you ran for the merged_gene_lists.txt. 

In [ ]:
## YOUR CODE HERE ## 
 
! grep "GO:0008380" GO_0008380_merged.enrichments.tsv

### QUESTION 3e

What would happen if we added random genes from the human genome to the gene set? 

Re-run the GORilla analysis for randomized_genes.txt using the **find_enrichment.py** script. Run the grep command to search for GO term "GO:0008380"


In [ ]:
## YOUR CODE HERE ##  
! find_enrichment.py randomized_genes.txt hg19.names.txt hg19.assocs --outfile GO_0008380_randomized.enrichments.tsv --pval_field=fdr_bh

In [ ]:
! grep "GO:0008380" GO_0008380_randomized.enrichments.tsv

What is the FDR p-value for GO term "GO:0008380"? 

ANSWER: 

